In [ ]:
#This notebook is by Anastasia Ruzmaikina for Kaggle Competition Natural Language Processing with Disaster Tweets

Twitter has become an important communication channel in times of emergency. The ubiquitousness of smartphones enables people to announce an emergency they’re observing in real-time. Because of this, more agencies are interested in programatically monitoring Twitter (i.e. disaster relief organizations and news agencies).

But, it’s not always clear whether a person’s words are actually announcing a disaster.

In this competition, you’re challenged to build a machine learning model that predicts which Tweets are about real disasters and which one’s aren’t. You’ll have access to a dataset of 10,000 tweets that were hand classified.

In this notebook, I use Llama-2-7b-hf to classify tweets as disaster tweets and non-disaster tweets. This notebook is 80.6% accurate on the validation dataset not previously seen by the model.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Install package for inferences
!pip install -qq --no-deps /kaggle/input/daigt-pip/peft-0.6.0-py3-none-any.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/daigt-pip/transformers-4.35.0-py3-none-any.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/daigt-pip/tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/accelerate-and-bitsandbytes/accelerate-0.29.3-py3-none-any.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/accelerate-and-bitsandbytes/bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/daigt-pip/optimum-1.14.0-py3-none-any.whl --use-deprecated=legacy-resolver
#!pip install -qq --no-deps /kaggle/input/llm-detect-pip/accelerate-0.24.1-py3-none-any.whl
#!pip install -qq --no-deps /kaggle/input/llm-detect-pip/bitsandbytes-0.41.1-py3-none-any.whl
#!os.register_at_fork()
!pip install -qq --no-dep /kaggle/input/bitsandbytes-accelerate/packages/accelerate-1.0.1-py3-none-any.whl --use-deprecated=legacy-resolver
!pip install -qq --no-dep /kaggle/input/bitsandbytes-accelerate/packages/bitsandbytes-0.42.0-py3-none-any.whl --use-deprecated=legacy-resolver


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

ambiguous option: --no-dep (--no-dependencies, --no-deps?)

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

ambiguous option: --no-dep (--no-dependencies, --no-deps?)


In [3]:
#!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/ --use-deprecated=legacy-resolver
#!pip install -q -U accelerate --no-index --find-links ../input/llm-detect-pip/ --use-deprecated=legacy-resolver
#!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/ --use-deprecated=legacy-resolver
#!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/ --use-deprecated=legacy-resolver
# Install package for inferences
#!pip install -qq --no-deps /kaggle/input/hf-libraries/bitsandbytes --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/daigt-pip/peft-0.6.0-py3-none-any.whl --use-deprecated=legacy-resolver
#!pip install -qq --no-deps /kaggle/input/daigt-pip/transformers-4.35.0-py3-none-any.whl 
#!pip install -qq --no-deps /kaggle/input/daigt-pip/tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -qq --no-deps /kaggle/input/daigt-pip/optimum-1.14.0-py3-none-any.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/llm-detect-pip/accelerate-0.24.1-py3-none-any.whl  --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/llm-detect-pip/bitsandbytes-0.41.1-py3-none-any.whl --use-deprecated=legacy-resolver
#!pip install -qq --no-deps /kaggle/input/bitsandbytes-0-42-0 --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/llama3-1-dependencies/dependencies/bitsandbytes-0.43.2-py3-none-manylinux_2_24_x86_64.whl  --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/llama3-1-dependencies/dependencies/accelerate-0.33.0-py3-none-any.whl  --use-deprecated=legacy-resolver

In [4]:
from __future__ import annotations

TARGET_MODEL = '/kaggle/input/llama2-7b-hf/Llama2-7b-hf' #'/kaggle/input/llama2-7b-hf/Llama2-7b-hf'#"mistralai/Mistral-7B-v0.1"

DEBUG = False

In [5]:
# ====================================================
# Directory settings
# ====================================================
from pathlib import Path

OUTPUT_DIR = Path("./")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

INPUT_DIR = Path("../input/")

In [6]:
import pandas as pd
train_df = pd.read_csv(INPUT_DIR / "/kaggle/input/nlp-getting-started/train.csv", sep=',')
test_df = pd.read_csv(INPUT_DIR / "/kaggle/input/nlp-getting-started/test.csv", sep=',')
external_df = pd.read_csv(INPUT_DIR / "/kaggle/input/nlp-getting-started/train.csv", sep=',')
train_df['input'] = 'TEXT: ' + train_df.text 
train_df['label'] = train_df.target
train_df.insert(1, 'input', train_df.pop('input'))
test_df['input'] = 'TEXT: ' + test_df.text 
test_df.insert(1, 'input', test_df.pop('input'))
#test_df['label'] = test_df.target
train_df

,id,input,keyword,location,text,target,label
0,1,TEXT: Our Deeds are the Reason of this #earthq...,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,1
1,4,TEXT: Forest fire near La Ronge Sask. Canada,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,1
2,5,TEXT: All residents asked to 'shelter in place...,NaN,NaN,All residents asked to 'shelter in place' are ...,1,1
3,6,"TEXT: 13,000 people receive #wildfires evacuat...",NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,1
4,7,TEXT: Just got sent this photo from Ruby #Alas...,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,1
...,...,...,...,...,...,...,...
7608,10869,TEXT: Two giant cranes holding a bridge collap...,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,1
7609,10870,TEXT: @aria_ahrary @TheTawniest The out of con...,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,1
7610,10871,TEXT: M1.94 [01:04 UTC]?5km S of Volcano Hawai...,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,1
7611,10872,TEXT: Police investigating after an e-bike col...,NaN,NaN,Police investigating after an e-bike collided ...,1,1


In [7]:
# rename column generated to label

#train_df = train_df.rename(columns={'label': 'label'})
#test_df = test_df.rename(columns={'label': 'label'})
#external_df = external_df.rename(columns={'label': 'label'})
test_df = test_df.drop(['id','keyword', 'location','text'], axis=1)
test_df.reset_index(inplace=True, drop=True)
test_df

,input
0,TEXT: Just happened a terrible car crash
1,TEXT: Heard about #earthquake is different cit...
2,"TEXT: there is a forest fire at spot pond, gee..."
3,TEXT: Apocalypse lighting. #Spokane #wildfires
4,TEXT: Typhoon Soudelor kills 28 in China and T...
...,...
3258,TEXT: EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY...
3259,TEXT: Storm in RI worse than last hurricane. M...
3260,TEXT: Green Line derailment in Chicago http://...
3261,TEXT: MEG issues Hazardous Weather Outlook (HW...


In [8]:
#train_df = pd.concat([train_df, external_df])
train_df = train_df.drop(['id','keyword', 'location','target','text'], axis=1)
train_df.reset_index(inplace=True, drop=True)
print(f"Train dataframe has shape: {train_df.shape}")
train_df.head()

Train dataframe has shape: (7613, 2)


,input,label
0,TEXT: Our Deeds are the Reason of this #earthq...,1
1,TEXT: Forest fire near La Ronge Sask. Canada,1
2,TEXT: All residents asked to 'shelter in place...,1
3,"TEXT: 13,000 people receive #wildfires evacuat...",1
4,TEXT: Just got sent this photo from Ruby #Alas...,1


In [9]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X = train_df.loc[:, train_df.columns != "label"]
y = train_df.loc[:, train_df.columns == "label"]

for i, (train_index, valid_index) in enumerate(skf.split(X, y)):
    train_df.loc[valid_index, "fold"] = i
    
print(train_df.groupby("fold")["label"].value_counts())
train_df.head()

fold  label
0.0   0        868
      1        655
1.0   0        869
      1        654
2.0   0        869
      1        654
3.0   0        868
      1        654
4.0   0        868
      1        654
Name: count, dtype: int64


,input,label,fold
0,TEXT: Our Deeds are the Reason of this #earthq...,1,2.0
1,TEXT: Forest fire near La Ronge Sask. Canada,1,0.0
2,TEXT: All residents asked to 'shelter in place...,1,0.0
3,"TEXT: 13,000 people receive #wildfires evacuat...",1,0.0
4,TEXT: Just got sent this photo from Ruby #Alas...,1,1.0


In [10]:
# fold0 as valid
valid_df = train_df[train_df["fold"] == 0]
train_df = train_df[train_df["fold"] != 0]
print(train_df.shape)
print(valid_df.shape)

(6090, 3)
(1523, 3)


In [11]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch

peft_config = LoraConfig(
    r=4, #
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    target_modules=[
        "q_proj",
        "v_proj"
    ],
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [12]:
from transformers import AutoTokenizer, LlamaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
base_model = LlamaForSequenceClassification.from_pretrained(
    TARGET_MODEL,
    num_labels=2,
    quantization_config=bnb_config,
    device_map={"":0}
)
base_model.config.pretraining_tp = 1 # 1 is 7b
base_model.config.pad_token_id = tokenizer.pad_token_id


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location=map

In [14]:
model = get_peft_model(base_model, peft_config)

In [15]:
model.print_trainable_parameters()

trainable params: 2,105,344 || all params: 6,609,457,152 || trainable%: 0.031853508564813525


In [16]:
#remove this for actual calculation  here we take a smaller sample of the dataframe to speed up 
train_df = train_df.sample(2000, random_state=42)  #2400
print(train_df.tail())
print(train_df.label.value_counts(), valid_df.label.value_counts())

                                                  input  label  fold
4111  TEXT: Hail:The user summons a hailstorm lastin...      0   1.0
997   TEXT: Mopheme and Bigstar Johnson are a proble...      0   1.0
3012  TEXT: The answer my friend is yelling in the w...      0   3.0
1317  TEXT: Now playing: Boat Club - Memories on Lon...      0   4.0
2048  TEXT: The girl that I wanna save is like a dan...      0   2.0
label
0    1170
1     830
Name: count, dtype: int64 label
0    868
1    655
Name: count, dtype: int64


In [17]:
valid_df1 = valid_df.sample(frac = 0.04, random_state=42)
valid_df = valid_df.loc[~valid_df.index.isin(valid_df1.index)]
print(train_df.shape)
print(valid_df.shape)
print(valid_df1.shape)

(2000, 3)
(1462, 3)
(61, 3)


In [18]:
# datasets
from datasets import Dataset

# from pandas
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)
test_ds = Dataset.from_pandas(test_df)
valid_ds1 = Dataset.from_pandas(valid_df1)


In [19]:
def preprocess_function(examples, max_length=512):
    return tokenizer(examples["input"], truncation=True, max_length=max_length, padding=True)

In [20]:
train_tokenized_ds = train_ds.map(preprocess_function, batched=True)
valid_tokenized_ds = valid_ds.map(preprocess_function, batched=True)
test_tokenized_ds = test_ds.map(preprocess_function, batched=True)
valid_tokenized_ds1 = valid_ds1.map(preprocess_function, batched=True)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1462 [00:00<?, ? examples/s]

Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

In [21]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

In [22]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy_val = accuracy_score(labels, predictions)
    #roc_auc_val = roc_auc_score(labels, predictions)
    
    return {
        "accuracy": accuracy_val,
        #"roc_auc": roc_auc_val,
    }

In [23]:
from transformers import TrainingArguments, Trainer

steps = 5 if DEBUG else 20

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=5e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    max_grad_norm=0.3,
    optim='paged_adamw_32bit',
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    push_to_hub=False,
    warmup_steps=steps,
    eval_steps=steps,
    logging_steps=steps,
    report_to='none' # if DEBUG else 'wandb',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_ds,
    eval_dataset=valid_tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
20,1.017200,0.663574,0.748974
40,0.705800,0.583984,0.776334
60,0.659000,0.616699,0.796170
80,0.588600,0.624023,0.768810
100,0.540900,0.601074,0.790698
120,0.608800,0.545898,0.806430


TrainOutput(global_step=125, training_loss=0.6763491878509521, metrics={'train_runtime': 7734.8496, 'train_samples_per_second': 0.259, 'train_steps_per_second': 0.016, 'total_flos': 7229877829632000.0, 'train_loss': 0.6763491878509521, 'epoch': 1.0})

In [24]:
#this is to check if the predictions work on a known dataframe not previously seen by the model
preds = trainer.predict(valid_tokenized_ds1)
logits = preds.predictions


In [25]:
# from scipy.special import expit as sigmoid
import numpy as np
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  
logits = sigmoid(logits)
logits = np.argmax(logits, axis=1)
print(logits)
logits.shape, logits[0:5]

[0 0 0 1 0 1 0 1 1 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 0 1 1 0 1
 0 1 1 1 1 1 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 1]


((61,), array([0, 0, 0, 1, 0]))

In [26]:
#this is to check if the predictions work on a known dataframe not previosly seen by the model
valid_df1['preds'] = logits #np.argmax(probs, axis=1)
valid_df1

,input,label,fold,preds
2686,TEXT: Ignition Knock (Detonation) Sensor-Senso...,1,0.0,0
5465,TEXT: Reddit Will Now Quarantine Offensive Con...,0,0.0,0
7283,TEXT: reap the whirlwind,0,0.0,0
5781,TEXT: Twitter is doing a count down to the rio...,1,0.0,1
4976,TEXT: #3: TITAN WarriorCord 100 Feet - Authent...,0,0.0,0
...,...,...,...,...
4711,TEXT: Perfect night for a soak! Lava here I co...,0,0.0,1
2186,TEXT: MH370: Aircraft debris found on La Reuni...,1,0.0,1
78,TEXT: I-77 Mile Marker 31 to 40 South Mooresvi...,1,0.0,1
435,TEXT: @_Doofus_ @diamorfiend im jokin still ca...,0,0.0,0


In [27]:
#this is for the predictions on the test set
preds = trainer.predict(test_tokenized_ds)#.predictions.astype(float)
logits = preds.predictions


In [28]:
logits = preds.predictions
logits

array([[ 0.9897 ,  2.531  ],
       [ 1.875  ,  1.618  ],
       [ 0.6772 ,  3.678  ],
       ...,
       [-4.46   ,  0.953  ],
       [-4.793  , -0.04187],
       [-6.94   ,  2.008  ]], dtype=float16)

In [29]:
# from scipy.special import expit as sigmoid
import numpy as np
def sigmoid(x):
    #if x > -100:
        return 1 / (1 + np.exp(-x))  
   # else:
       # return 0
logits = sigmoid(logits)
logits = np.argmax(logits, axis=1)
logits[:100]

array([1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1])

In [30]:
logits.shape, logits[0:5]


((3263,), array([1, 0, 1, 1, 1]))

In [31]:
#this is to provide the sample submission
sub = pd.DataFrame()
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

sub['id'] = test_df['id']
import math

sub["prediction"] = logits

sub['prediction'] = sub['prediction'].astype('int')
sub.to_csv('/kaggle/working/submission.csv', index=False)
sub[:100]

,id,prediction
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1
...,...,...
95,323,0
96,324,0
97,325,0
98,326,1
